### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# B2C Automated Classification Accelerator 

## Step 1: Create Conda Enviorment 
It is needed to create an enviorment where all the required python dependencies are available.

<i>If conda enviorment already created then assign that path in</i> **CONDA_ENV_PATH** <i>env variable</i>

In [ ]:
!pip install -r ./../requirements.txt

#### 1. Setup conda enviornment

In [ ]:
import shutil
CONDA_ENV_NAME = "b2c_env"

In [ ]:
!rm -rf ./conda

In [ ]:
!odsc conda create --file ./environment.yml --name {CONDA_ENV_NAME} --slug {CONDA_ENV_NAME}_slug

In [ ]:
env CONDA_PREFIX=/home/datascience/conda/b2c_env_slug

In [ ]:
"""
This is need because moder_artifact.build tries to get manifest file from conda directory default: /opt/conda.

But it does not have mainifest file. Hence, it throws following error while building the model artifact.

Exception: Could not locate manifest file in the provided conda environment: /opt/conda. Dir Listing - ['ssl', 'lib', 'envs', 'conda-meta', 'x86_64-conda_cos6-linux-gnu', 'include', 'condabin', 'LICENSE.txt', 'share', 'bin', 'etc', 'x86_64-conda-linux-gnu', 'shell', 'compiler_compat', 'pkgs', 'cx_Oracle-doc']

"""

import os
os.environ.get("CONDA_PREFIX", None)

#### 2. Set required enviornment variables

In [ ]:
from training_scripts.utils import set_config
set_config()

In [ ]:
import ads
import logging
import os
import datetime

from dateutil.tz import tzutc

from training_scripts.constants import COMPARTMENT_ID, BUCKET_URL, PROJECT_ID, SUBNET_ID, LOG_GROUP_ID, \
    CONDA_ENV_PATH, LOGGING_FORMAT, VERSION
from training_scripts.jobs import model_artifact
from training_scripts.utils.job_utility import get_infrastructure, create_job, execute_job

import oci
signer = oci.auth.signers.get_resource_principals_signer()

In [ ]:
os.getenv(BUCKET_URL)

In [ ]:
object_storage = oci.object_storage.ObjectStorageClient(config={},
                                                                         signer=signer)
namespace = object_storage.get_namespace().data
namespace

In [ ]:
from training_scripts.jobs.conda_environment import get_conda_env

In [ ]:
conda_env = get_conda_env()

In [ ]:
PYTHON_ENV, PYTHON_BUCKET = conda_env

In [ ]:
PYTHON_BUCKET

In [ ]:
os.environ.setdefault(CONDA_ENV_PATH,
                          f"oci://{PYTHON_BUCKET}")
os.environ.get(CONDA_ENV_PATH, None)

In [ ]:
import ads
ads.set_auth(auth='resource_principal')

### Create Log Group

In [ ]:
from training_scripts.utils.job_utility import create_log, get_log_group_ocid
import uuid

In [ ]:
group_log_name = f"ai4service-group-log-{str(uuid.uuid4())}"
group_log_name

In [ ]:
response = create_log(name=group_log_name, group_log_id=None)
response.status

In [ ]:
log_group_ocid = get_log_group_ocid(name=group_log_name)
os.environ[LOG_GROUP_ID] = log_group_ocid
os.getenv(LOG_GROUP_ID)

#### 3. Run Ingestion Jobs

In [ ]:
os.getcwd()

In [ ]:
current_time = datetime.datetime.now(tz=tzutc())
infrastructure = get_infrastructure()

In [ ]:
infrastructure

In [ ]:
ingestion_job = create_job(f"ingest-data-{os.getenv(VERSION)}",infrastructure, current_time)

In [ ]:
ingestion_job

In [ ]:
execute_job(ingestion_job, False)


In [ ]:
build_model_job = create_job(f"build-model-{os.getenv(VERSION)}", infrastructure, current_time)

In [ ]:
build_model_job

In [ ]:
execute_job(build_model_job, False)


#### 4. Build B2C MultiTailClassifier Model Artifact

In [ ]:
# Optional: TODO Have to figure out how to get latest model id from oci datascience api
# model = model_artifact.build()

In [ ]:
# model

In [ ]:
model_id = model.id # Provide latest model id here
print(f"Model OCID: {model_id}")

In [ ]:
from training_scripts.utils.job_utility import get_log_ocid


In [ ]:
access_log_name = "ai4service-access-log-"+str(uuid.uuid4())
response = create_log(access_log_name, group_log_id=log_group_ocid)
print(response.status)
access_log_id = get_log_ocid(name=access_log_name, group_log_id=log_group_ocid)
access_log_id

In [ ]:
predict_log_name = "ai4service-predict-log-"+str(uuid.uuid4())
response = create_log(predict_log_name, group_log_id=log_group_ocid)
print(response.status)
predict_log_id = get_log_ocid(name=predict_log_name, group_log_id=log_group_ocid)
predict_log_id

### Initialize ModelDeploymentProperties

##### Replace INSTANCE_SHAPE with one of the following shape name as per requirements

|  | Shape name | Number of OCPUs | Memory (GB) | Details |
|---|---|---|---|---|
|  | VM.Standard2.1 | 1 | 15 | VM.Standard2.1 |
|  | VM.Standard2.2 | 2 | 30 | VM.Standard2.2 |
|  | VM.Standard2.4 | 4 | 60 | VM.Standard2.4 |
|  | VM.Standard2.8 | 8 | 120 | VM.Standard2.8 |
|  | VM.Standard2.16 | 16 | 240 | VM.Standard2.16 |
|  | VM.Standard2.24 | 24 | 320 |  VM.Standard2.24 |

In [ ]:
from ads.model.deployment import ModelDeploymentProperties

model_deployment_properties = ModelDeploymentProperties(
    model_id
).with_prop(
    'display_name', f"Incident Classification Model Deployment {os.getenv(VERSION)}"
).with_prop(
    "project_id", os.getenv(PROJECT_ID)
).with_prop(
    "compartment_id", os.getenv(COMPARTMENT_ID)
).with_logging_configuration(
    os.getenv(LOG_GROUP_ID), access_log_id, os.getenv(LOG_GROUP_ID), predict_log_id
).with_instance_configuration(
    config={"INSTANCE_SHAPE":"VM.Standard2.4", "INSTANCE_COUNT":"1",'bandwidth_mbps':10}
)


In [ ]:
from ads.model.deployment import ModelDeployer
deployer = ModelDeployer()
deployment = deployer.deploy(model_deployment_properties)

In [ ]:
f"Deployment {deployment.model_deployment_id} is {deployment.state.name}"